<a href="https://colab.research.google.com/github/odacostaz/GameOfLife/blob/main/Modelo_de_Proyecciones_Asset_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Paquetes usados

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd # Para manipulación de datos 
import numpy as np # Biblioteca numérica
from sklearn.model_selection import train_test_split # Para separar datos de entrenamiento y prueba
from sklearn.linear_model import  LinearRegression # Para hacer la regresión linear
from sklearn.metrics import r2_score # Coeficiente de determinación
from sklearn.metrics import mean_squared_error # Error cuadrático medio
import matplotlib.pyplot as plt # Gráficos
import seaborn as sns # Gráficos estadísticos

#Preprocesamiento de los datos

In [ ]:
# Importación de base de datos
data = pd.read_excel("DATA.xlsx", sheet_name= "SALIDA FINAL")
data

FileNotFoundError: ignored

In [ ]:
#Numero de datos
data.shape

In [ ]:
# Dataset final -- A modo de series de tiempo
data.index = pd.Index(data["Fecha"])
data.drop("Fecha", axis = 1, inplace = True)
print(data)

#Pronostico de variables macro

In [ ]:
# Importación de pronosticos
macro = pd.read_excel("DATA.xlsx", sheet_name= "SALIDA MACRO", index_col=0)
macro

In [ ]:
#PRONOSTICOS FIN DE AÑO
pron_fin = ["P_IPC", "P_ISE", "P_BR", "P_PT", "P_FED", "P_PG", "P_IPC3", "P_BR6", "P_FED6", "P_PG6", "P_TRM"]

for i in range(0,11):
  exec("%s = %f" % (pron_fin[i], macro.iloc[0,i]))

In [ ]:
#PRONOSTICOS 12 meses
pron_12m = ["P_IPC_12", "P_ISE_12", "P_BR_12", "P_PT_12", "P_FED_12", "P_PG_12", "P_IPC3_12", "P_BR6_12", "P_FED6_12", "P_PG6_12", "P_TRM_12"]

for i in range(0,11):
  exec("%s = %f" % (pron_12m[i], macro.iloc[1,i]))

#Pre-parametrización del modelo

In [ ]:
#Separamos en la matriz de variables y regresoras
variables = data.iloc[:,0:19]
variables.drop("Vista", axis = 1, inplace = True)
regresoras = data.iloc[:,22:]
regresoras["TES MP"] = data["TES MP"]

In [ ]:
m_col = ['ISE Mensual %', 'Brent (USD)']
m_pei = ['IPC anual %', 'ISE Mensual %']
m_deuda = ['Inflación T+3','BanRep T+6','TES MP']
m_tes = ['Tasa FED %','Inflación T+3','BanRep T+6']
m_ag = ['FED T+6', 'PIB Global T+6']
m_vg = ['Tasa FED %', 'Brent (USD)']

In [ ]:
#Definimos los pronosticos macro fin año
x_pred = pd.DataFrame(np.array([[P_IPC, P_ISE, P_BR, P_FED, P_PG, P_PT, P_IPC3, P_BR6, P_FED6, P_PG6, 0 ]]), columns= regresoras.columns)
x_pred

In [ ]:
#Definimos los pronosticos 12 meses
x_pred_12m = pd.DataFrame(np.array([[P_IPC_12, P_ISE_12, P_BR_12, P_FED_12, P_PG_12, P_PT_12, P_IPC3_12, P_BR6_12, P_FED6_12, P_PG6_12, 0 ]]), columns= regresoras.columns)
x_pred_12m

In [ ]:
#Creamos una matriz para alojar las proyecciones fin de año de los assets
varcop = np.array(["Acc G COP",	"RF G COP",	"Vista USD COP"])
varnor = np.array(variables.columns)
columnas = np.append(varnor,varcop)
columnas

forecast_prices = pd.DataFrame(np.array([range(21)]), columns= columnas)
forecast_prices

In [ ]:
#Creamos una matriz para alojar las proyecciones 12 meses de los assets
forecast_prices_12m = pd.DataFrame(np.array([range(21)]), columns= columnas)
forecast_prices_12m

# Modelo

In [ ]:
#COLCAP
y = variables.iloc[:,0]
x = regresoras[m_col]
modelo = LinearRegression()
modelo.fit(x, y)
y_pred = modelo.predict(x_pred[m_col])
forecast_prices[forecast_prices.columns[0]] = y_pred
#12 meses
y_pred = modelo.predict(x_pred_12m[m_col])
forecast_prices_12m[forecast_prices_12m.columns[0]] = y_pred

In [ ]:
#PEI
y = variables.iloc[:,1]
x = regresoras[m_pei]
modelo = LinearRegression()
modelo.fit(x, y)
y_pred = modelo.predict(x_pred[m_pei])
forecast_prices[forecast_prices.columns[1]] = y_pred
#12 meses
y_pred = modelo.predict(x_pred_12m[m_pei])
forecast_prices_12m[forecast_prices_12m.columns[1]] = y_pred

In [ ]:
#TES
for i in range(2,6):
  y = variables.iloc[:,i]
  x = regresoras[m_tes]
  modelo = LinearRegression()
  modelo.fit(x, y)  
  y_pred = modelo.predict(x_pred[m_tes])
  forecast_prices[forecast_prices.columns[i]] = y_pred
  y_pred = modelo.predict(x_pred_12m[m_tes])                #12 meses
  forecast_prices_12m[forecast_prices_12m.columns[i]] = y_pred

In [ ]:
#Incluimos TES MP en x_pred
x_pred["TES MP"] = forecast_prices["TES MP"]
x_pred_12m["TES MP"] = forecast_prices_12m["TES MP"]  #12 meses

In [ ]:
#Deuda
for i in range(6,15):
  y = variables.iloc[:,i]
  x = regresoras[m_deuda]
  modelo = LinearRegression()
  modelo.fit(x, y)
  y_pred = modelo.predict(x_pred[m_deuda])
  forecast_prices[forecast_prices.columns[i]] = y_pred
  y_pred = modelo.predict(x_pred_12m[m_deuda])            #12 meses
  forecast_prices_12m[forecast_prices_12m.columns[i]] = y_pred

In [ ]:
#Activos Internacionales
for i in range(15,17):
  y = variables.iloc[:,i]
  x = regresoras[m_ag]
  modelo = LinearRegression()
  modelo.fit(x, y)
  y_pred = modelo.predict(x_pred[m_ag])
  forecast_prices[forecast_prices.columns[i]] = y_pred
  y_pred = modelo.predict(x_pred_12m[m_ag])               #12 meses
  forecast_prices_12m[forecast_prices_12m.columns[i]] = y_pred

#Vista DOLAR
y = variables.iloc[:,17]
x = regresoras[m_vg]
modelo = LinearRegression()
modelo.fit(x, y)
y_pred = modelo.predict(x_pred[m_vg])
forecast_prices[forecast_prices.columns[17]] = y_pred
y_pred = modelo.predict(x_pred_12m[m_vg])                #12 meses
forecast_prices_12m[forecast_prices_12m.columns[17]] = y_pred

In [ ]:
#Pesificamos las proyecciones de activos globales
forecast_prices['Acc G COP'] = forecast_prices['Acc G USD'] * P_TRM
forecast_prices['RF G COP'] = forecast_prices['RF G USD'] * P_TRM
forecast_prices['Vista USD COP'] = forecast_prices['Vista USD'] * P_TRM

#12 meses
forecast_prices_12m['Acc G COP'] = forecast_prices_12m['Acc G USD'] * P_TRM_12
forecast_prices_12m['RF G COP'] = forecast_prices_12m['RF G USD'] * P_TRM_12
forecast_prices_12m['Vista USD COP'] = forecast_prices_12m['Vista USD'] * P_TRM_12

In [ ]:
forecast_prices

In [ ]:
forecast_prices_12m

#Cálculo de retornos proyectados

##Fin de año

In [ ]:
#Proyección de rentabilidades a fin de año
#Creamos el data frame
forescast_return_EA = pd.DataFrame(np.array([range(21)]), columns= columnas)
forescast_return_EA

In [ ]:
#Calculamos los retornos proyectados a fin de año
pos_fin_año = 60
pos_fin_año = pos_fin_año -1
nombres_col = forescast_return_EA.columns

for i in range(21):
  x = nombres_col[i]
  forescast_return_EA[x] = np.log(forecast_prices[x]/data[x].iloc[pos_fin_año])

In [ ]:
forescast_return_EA

In [ ]:
#Rentabilidad corrida en el año
#Creamos el data frame
forescast_ret_corrido = pd.DataFrame(np.array([range(21)]), columns= columnas)
forescast_ret_corrido

In [ ]:
#Calculamos los retornos corridos en el año
pos_año_corrido = len(data)
pos_año_corrido = pos_año_corrido -1

nombres_col = forescast_ret_corrido.columns

for i in range(21):
  x = nombres_col[i]
  forescast_ret_corrido[x] = np.log(data[x].iloc[pos_año_corrido]/data[x].iloc[pos_fin_año])

In [ ]:
forescast_ret_corrido

In [ ]:
#Rentabilidad faltante en el año
#Creamos el data frame
ret_faltante = pd.DataFrame(np.array([range(21)]), columns= columnas)
ret_faltante

In [ ]:
#Calculamos el retorno faltante del año
nombres_col = ret_faltante.columns

for i in range(21):
  x = nombres_col[i]
  ret_faltante[x] = forescast_return_EA[x]-forescast_ret_corrido[x]

In [ ]:
ret_faltante

## 12 meses

In [ ]:
#Proyección de rentabilidades a fin de año
#Creamos el data frame
forescast_return_12m = pd.DataFrame(np.array([range(21)]), columns= columnas)
forescast_return_12m

In [ ]:
#Calculamos los retornos proyectados a 12 meses
pos_mes_actual = len(data)
pos_mes_actual = pos_mes_actual -1
nombres_col_12m = forescast_return_12m.columns

for i in range(21):
  x = nombres_col[i]
  forescast_return_12m[x] = np.log(forecast_prices_12m[x]/data[x].iloc[pos_mes_actual])

In [ ]:
forescast_return_12m

#Descarga de proyecciones

In [ ]:
Base = pd.concat([forecast_prices, forescast_return_EA, forescast_ret_corrido, ret_faltante,forecast_prices_12m,forescast_return_12m])
Base.index = ["Precio proyectado Fin Año", "Rentabilidad EA Fin Año", "Rentabilidad año corrido", "Rentabilidad restante Fin Año","Precio proyectado 12 meses", "Rentabilidad EA 12 meses"]
Base

In [ ]:
Base.to_excel("Proyecciones final.xlsx")